<a href="https://colab.research.google.com/github/VladislavKlekovkin/s1mulation/blob/master/1_cross_entropy_method_random_start_agent_position.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
!git clone https://github.com/VladislavKlekovkin/s1mulation.git 
sys.path.append(os.path.join('./', 's1mulation'))

Cloning into 's1mulation'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 44 (delta 21), reused 23 (delta 8), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [2]:
from environment import Space

#import gym
from collections import namedtuple
import numpy as np
from torch.utils.tensorboard import SummaryWriter


import torch
import torch.nn as nn
from torch import optim

WIDTH = 8
HEIGHT = 8

HIDDEN_SIZE = 128
BATCH_SIZE = 64
PERCENTILE = 70

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
env = Space(width=WIDTH, height=HEIGHT)
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.width * env.height
n_actions = len(env.action_space)

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 0.9:
        print("Solved!")
        break
writer.close()

Evn initializing
0: loss=1.386, reward_mean=0.1, reward_bound=0.1
1: loss=1.382, reward_mean=0.1, reward_bound=0.1
2: loss=1.384, reward_mean=0.1, reward_bound=0.1
3: loss=1.379, reward_mean=0.1, reward_bound=0.1
4: loss=1.382, reward_mean=0.1, reward_bound=0.1
5: loss=1.384, reward_mean=0.1, reward_bound=0.1
6: loss=1.381, reward_mean=0.1, reward_bound=0.1
7: loss=1.364, reward_mean=0.1, reward_bound=0.1
8: loss=1.370, reward_mean=0.1, reward_bound=0.1
9: loss=1.367, reward_mean=0.1, reward_bound=0.2
10: loss=1.370, reward_mean=0.1, reward_bound=0.1
11: loss=1.355, reward_mean=0.1, reward_bound=0.1
12: loss=1.356, reward_mean=0.1, reward_bound=0.1
13: loss=1.347, reward_mean=0.1, reward_bound=0.1
14: loss=1.345, reward_mean=0.1, reward_bound=0.1
15: loss=1.345, reward_mean=0.1, reward_bound=0.1
16: loss=1.335, reward_mean=0.1, reward_bound=0.1
17: loss=1.336, reward_mean=0.1, reward_bound=0.1
18: loss=1.331, reward_mean=0.1, reward_bound=0.1
19: loss=1.327, reward_mean=0.1, reward_bou

In [17]:
# testing. One game

from time import sleep
from IPython.display import clear_output

obs = env.reset()
sm = nn.Softmax(dim=1)

while True:
  clear_output(wait=True)
  sleep(1)

  for i in env.state:
    print(i)
  print('-' * 16)

  obs_v = torch.FloatTensor([obs])
  act_probs_v = sm(net(obs_v))
  act_probs = act_probs_v.data.numpy()[0]
  action = np.random.choice(len(act_probs), p=act_probs)
  next_obs, reward, is_done, _ = env.step(action)

  if is_done:
    print('Revard:', reward)
    break
  obs = next_obs

[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0.5, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
----------------
Revard: 1.0


In [22]:
torch.save({
            'epoch': iter_no,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_v,
            }, './model-cross-entropy-random-agent-start-position')

In [23]:
!ls -l

total 120
-rw-r--r-- 1 root root 110367 Oct 16 21:12 model-cross-entropy-random-agent-start-position
drwxr-xr-x 3 root root   4096 Oct 16 20:06 runs
drwxr-xr-x 4 root root   4096 Oct 16 20:06 s1mulation
drwxr-xr-x 1 root root   4096 Oct 14 16:31 sample_data


In [24]:
from google.colab import files
files.download('./model-cross-entropy-random-agent-start-position')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>